In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

In [ ]:
#function to split the data into training, validation and test sets, and calculate their Fourier coefficients along
#with the corresponding labels. At t=0 all of the signals are equal, so we omit the first timestep. Ntrain is the 
#number of trajectories in the training set, Nval is the number in the validation set and Ntest is the number in 
#the test set
def fouriertrainvaltest(X, Y, Ntrain, Nval, Ntest):
    
    #Generating a training set with Ntrain trajectories, a validation with Nval trajectories and a test set with
    #Ntest trajectories. At t=0 all of the signals are equal, so we omit the first timestep.
    Xtrain = X[0:Ntrain, 1:401]
    Xval = X[Ntrain:Ntrain+Nval, 1:401]
    Xtest = X[Ntrain+Nval:Ntrain+Nval+Ntest, 1:401]

    #extract the corresponding labels for the training, validation and test sets.
    Ytrain = Y[0:Ntrain, :]
    Yval = Y[Ntrain:Ntrain+Nval, :]
    Ytest = Y[Ntrain+Nval:Ntrain+Nval+Ntest, :]
    
    #calculating the Fourier coefficients for each subset.
    XtrainF = np.fft.fft(Xtrain)
    XvalF = np.fft.fft(Xval)
    XtestF = np.fft.fft(Xtest)
    
    #Prepare to split the Fourier coefficients into their real and imaginary components. Each complex number will 
    #occupy two columns: one for the real part and one for the imaginary part. Therefore, we create new arrays that 
    #have twice the number of columns. 
    xtrain = np.zeros((XtrainF.shape[0], 2*XtrainF.shape[1]))
    xval = np.zeros((XvalF.shape[0], 2*XvalF.shape[1]))
    xtest = np.zeros((XtestF.shape[0], 2*XtestF.shape[1]))

    #For each Fourier coefficient in the training set, split into real and imaginary parts. These parts are then
    #stored alternately (even indices for real, odd indices for imaginary).
    for i in range(XtrainF.shape[0]):
        for j in range(XtrainF.shape[1]):
            xtrain[i, 2*j] = XtrainF[i,j].real
            xtrain[i, 2*j + 1] = XtrainF[i,j].imag
        
    #Do the same for the test set, splitting the Fourier coefficients into their real and imaginary parts.
    for i in range(XtestF.shape[0]):
        for j in range(XtestF.shape[1]):
            xtest[i, 2*j] = XtestF[i,j].real
            xtest[i, 2*j + 1] = XtestF[i,j].imag
            
    #Similarly, split the Fourier coefficients for the validation set.
    for i in range(XvalF.shape[0]):
        for j in range(XvalF.shape[1]):
            xval[i, 2*j] = XvalF[i,j].real
            xval[i, 2*j + 1] = XvalF[i,j].imag
            
    #Return the transformed training, validation and test sets along with their corresponding labels
    return(xtrain, xval, xtest, Ytrain, Yval, Ytest)

In [ ]:
#Function to calculate the R-squared metric. This function takes the true and predicted values, and calculates the 
#R-squared value
def r_square(y, y_pred):
    residual = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
    total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
    r2 = 1 - residual/total
    return(r2)

In [ ]:
#Initalise arrays to store the loss values for each interval
testloss = np.zeros(10)
trainingloss = np.zeros(10)
validationloss = np.zeros(10)

In [ ]:
#Initialise the upper bound for the interval
upper_bound = 0.25

#Loop over 10 different values, adjusting the upper bound during each iteration
for i in tqdm(range(10)):
    
    #Load training data from csv files based on the current upper bound
    X = np.loadtxt('Data/Xtrainx_varyηandω_0.25and{0}.csv'.format(np.round(upper_bound, 2)), delimiter=',')
    Y = np.loadtxt('Data/Ytrain_varyηandω_0.25and{0}.csv'.format(np.round(upper_bound, 2)), delimiter=',')
    
    #Extract the values of $\omega_c$, $s$ and $\eta$ from Y for training
    Y_params = Y[:,3:6]
    
    #Scale the extracted parameters to a [0,1] range using min-max scaling
    scaler=MinMaxScaler()
    Y_scaled = scaler.fit_transform(Y_params)

    #Generating a training, validation and test set
    xtrain, xval, xtest, Ytrain, Yval, Ytest = fouriertrainvaltest(X, Y_scaled, 4800, 2400, 2400)
    
    #Create a sequential model
    model = tf.keras.Sequential()
    #Add the hidden layers
    model.add(tf.keras.layers.Dense(250, input_dim = (np.shape(xtrain)[1]), activation='sigmoid'))
    model.add(tf.keras.layers.Dense(250 , activation = 'sigmoid'))
    model.add(tf.keras.layers.Dense(250 , activation = 'sigmoid'))
    model.add(tf.keras.layers.Dense(250 , activation = 'sigmoid'))
    model.add(tf.keras.layers.Dense(250 , activation = 'sigmoid'))
    model.add(tf.keras.layers.Dense(80 , activation = 'sigmoid'))
    #Add the output layer
    model.add(tf.keras.layers.Dense(3, activation = 'linear'))

    #Setting the optimiser equal to the Adam optimiser with learning rate = 0.0001
    opt = tf.keras.optimizers.Adam(learning_rate = 0.0001)

    #Compliling the model
    model.compile(optimizer=opt, loss = 'mean_squared_error', metrics=[r_square])
    
    #Training the model
    history = model.fit(xtrain, Ytrain, epochs = 100000, validation_data = (xval, Yval), batch_size = np.shape(xtrain)[0], verbose=0)
    
    #Save the trained model weights
    model.save("Weights/training_regression_0.25and{0}.weights.h5".format(np.round(upper_bound, 2)))
               
    #Evaluate the model on the training, validation and test sets and store the loss metric
    trainingloss[i], trainingr2 = model.evaluate(xtrain, Ytrain)
    validationloss[i], validationr2 = model.evaluate(xval, Yval)
    testloss[i], testr2 = model.evaluate(xtest, Ytest)
               
    #Save the loss metrics to csv files
    np.savetxt('traininglossvintervallength.csv', trainingloss, delimiter=',')
    np.savetxt('validationlossvintervallength.csv', validationloss, delimiter=',')
    np.savetxt('testlossvintervallength.csv', testloss, delimiter=',')
               
    #Make predictions for the training validation, and test sets and revert scaling
    predictions_train = scaler.inverse_transform(model.predict(xtrain))
    predictions_val = scaler.inverse_transform(model.predict(xval))
    predictions_test = scaler.inverse_transform(model.predict(xtest))
               
    #Save predictions alongside the original index for training, validation and test sets to csv files
    np.savetxt('Predictions/predictionstrain_0.25and{0}.csv'.format(np.round(upper_bound, 2)), predictions_train, delimiter=',')
    np.savetxt('Predictions/predictionsval_0.25and{0}.csv'.format(np.round(upper_bound, 2)), predictions_val, delimiter=',')
    np.savetxt('Predictions/predictionstest_0.25and{0}.csv'.format(np.round(upper_bound, 2)), predictions_test, delimiter=',')

    #Increase the upper bound by 0.2 for the next iteration
    upper_bound += 0.2